In [1]:
import sys
import geopandas as gpd
import multiprocessing

sys.path.append('../utils')
import data_paths
from traffic_counts import TrafficCounts
from hbefa_hot_emissions import HbefaHotEmissions
from excel_calendar import Calendar

import inventory_processing

In [2]:
# import visum model
visum = gpd.read_file(data_paths.VISUM_FOLDER_PATH + 'visum_links.gpkg')
visum_red = visum.drop(visum.loc[(visum['road_type'] == 'none') | (visum['dtv_SUM']<1)].index, axis = 0)

visum_red.loc[visum_red['road_type']=='Access-residential', 'road_type'] = 'Local/Collector'
visum_red.loc[visum_red['road_type']=='Distributor/Secondary', 'road_type'] = 'TrunkRoad/Primary-National'

In [3]:
# cut visum model to the Munich area
munich_boarders = gpd.read_file(data_paths.MUNICH_BOARDERS_FILE).to_crs(25832)
visum_red = gpd.clip(visum_red, munich_boarders)

In [4]:
# initialize calendar
cal = Calendar()
# initialize traffic cycles
cycles = TrafficCounts()
# initialize HBEFA emission factors
hbefa = HbefaHotEmissions()

Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_PC.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_LCV.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_HGV.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_Coach.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_MOT.XLS


In [5]:
# get dates for normweekdays in 2019
dates = cal.get_calendar()
normweekdays_2019 = dates[(dates['day_type'] == 1) &
                          (dates['date'].between('2019-01-01','2019-12-31'))]

# calculate vehicle shares on average norm weekday in 2019
# these values are used to calculate the vehicle share correction factors
weekday_shares_2019 = cycles.vehicle_shares.loc[:,normweekdays_2019['date']].reset_index()
weekday_shares_2019 = weekday_shares_2019.groupby(['road_type','vehicle_class'])[0].mean()

In [6]:
visum_red['hgv_corr'] = visum_red.apply(lambda row : row['delta_HGV'] / weekday_shares_2019.loc[row['road_type'],'HGV'], axis = 1)
visum_red['lcv_corr'] = visum_red.apply(lambda row : row['delta_LCV'] / weekday_shares_2019.loc[row['road_type'],'LCV'], axis = 1)

In [7]:
#NUMBER_OF_PROCESSES = multiprocessing.cpu_count()
NUMBER_OF_PROCESSES = 6
df = cal.get_calendar()
dates = [d.strftime("%Y-%m-%d") for d in df[df['date'].between('2019-01-01','2019-12-31')]['date']]

visum_red = visum_red.reset_index(drop = True).reset_index()

parameters = [(d, visum_red.to_dict('records'), cycles, hbefa) for d in dates]

with multiprocessing.Pool(NUMBER_OF_PROCESSES) as pool:
    res = pool.starmap(inventory_processing.calculate_daily_co2_emissions, parameters)

Cannot process 2019-01-01
Error: 'Local/Collector'
Finished 2019-01-02
Finished 2019-01-17
Finished 2019-02-02
Finished 2019-02-18
Finished 2019-03-06
Finished 2019-01-03
Finished 2019-03-22
Finished 2019-01-18
Finished 2019-02-03
Finished 2019-02-19
Finished 2019-03-07
Finished 2019-01-04
Finished 2019-03-23
Finished 2019-01-19
Finished 2019-02-04
Finished 2019-02-20
Finished 2019-03-08
Finished 2019-01-05
Finished 2019-03-24
Finished 2019-01-20
Finished 2019-02-05
Finished 2019-02-21
Finished 2019-03-09
Finished 2019-01-06
Finished 2019-03-25
Finished 2019-01-21
Finished 2019-02-06
Finished 2019-02-22
Finished 2019-03-10
Finished 2019-01-07
Finished 2019-01-22
Finished 2019-03-26
Finished 2019-02-07
Finished 2019-02-23
Finished 2019-03-11
Finished 2019-01-08
Finished 2019-01-23
Finished 2019-03-27
Finished 2019-02-08
Finished 2019-02-24
Finished 2019-03-12
Finished 2019-01-09
Finished 2019-03-28
Finished 2019-01-24
Finished 2019-02-09
Finished 2019-02-25
Finished 2019-03-13
Finished 

In [10]:

visum_em = visum_red
visum_em['emissions'] = 0
visum_em

,index,road_link_id,road_type,hour_capacity,speed,dtv_SUM,delta_PC,delta_LCV,delta_HGV,SLOPE,geometry,hgv_corr,lcv_corr,emissions
0,0,33564,Local/Collector,800,30,332.67,0.853699,0.132263,0.015030,0.56,"LINESTRING (696496.643 5329119.087, 696477.780...",0.495783,1.322712,0
1,1,33564,Local/Collector,800,30,329.12,0.865946,0.121536,0.012154,-0.56,"LINESTRING (696477.780 5329125.673, 696496.643...",0.400905,1.215435,0
2,2,52816430,Local/Collector,800,30,329.12,0.865946,0.121536,0.012154,-0.28,"LINESTRING (696496.643 5329119.087, 696559.768...",0.400905,1.215435,0
3,3,52816430,Local/Collector,800,30,332.67,0.853699,0.132263,0.015030,0.28,"LINESTRING (696559.768 5329147.956, 696496.643...",0.495783,1.322712,0
4,4,42114,Local/Collector,800,30,443.52,0.789141,0.198413,0.013528,0.12,"LINESTRING (696823.867 5329142.558, 696809.564...",0.446245,1.984246,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34389,34389,52832035,Local/Collector,500,20,572.87,0.911201,0.073315,0.015710,-0.24,"LINESTRING (687432.738 5344391.360, 687497.149...",0.518229,0.733195,0
34390,34390,36065,Local/Collector,500,20,213.88,0.874322,0.102861,0.023378,0.20,"LINESTRING (687610.332 5344755.453, 687581.861...",0.771144,1.028676,0
34391,34391,36065,Local/Collector,500,20,208.46,0.887460,0.095942,0.014391,-0.20,"LINESTRING (687381.874 5344468.682, 687407.330...",0.474716,0.959474,0
34392,34392,46573,TrunkRoad/Primary-National,1320,70,9226.02,0.853456,0.102103,0.044440,-1.06,"LINESTRING (688439.720 5344979.442, 688439.818...",0.787354,1.055031,0


In [ ]:

visum_em = visum_red
visum_em['emissions'] = 0



for item in res:
    if item == 0:
        continue
    else:
        for key, value in item.items():
            print(key)
            print(value)
            co2 = 0
            for vehicle in ['PC', 'LCV', 'HGV', 'BUS', 'MOT']:
                c = value[(vehicle, 'NOx')]
                if (c != c):
                    continue
                else:
                    co2 += c
            print(co2)
            break
    break                    
print(co2*1e-9)